# RECOMMENDATION SYSTEM

A recommendation system is a technique used to suggest relevant items to users based on their preferences, behavior, or similarities with other users. In the context of books, the goal is to recommend books a user is likely to enjoy based on available data.

Broadly, recommendation systems can be grouped into three main types.

---

## 1. Content-Based Recommendation System

A content-based recommendation system suggests items **based on the characteristics of the items themselves** and the user’s past preferences.

### How it works
- Each item (book) is described using features such as:
  - Genre
  - Author
  - Keywords
  - Description or summary
- A user profile is built from the items the user has liked or rated highly.
- The system recommends new items that are **similar in content** to what the user has already interacted with.

### Example
If a user likes books written by *Tom Clancy* or books tagged as *thriller*, the system will recommend other books with similar authors, genres, or themes.

### Advantages
- Does not require data from other users
- Works well for users with unique tastes
- No cold-start problem for items (as long as metadata exists)

### Limitations
- Limited diversity in recommendations
- Cannot recommend items outside the user’s established interests
- Requires rich and clean item metadata

---

## 2. Collaborative Filtering Recommendation System

Collaborative filtering recommends items based on **user behavior patterns** rather than item content.

### How it works
- Uses historical interactions such as ratings or clicks
- Finds similarities between:
  - Users (user-based filtering), or
  - Items (item-based filtering)
- Assumes that users who behaved similarly in the past will have similar preferences in the future

### Example
If many users who liked *The Juror* also liked *The Little Drummer Girl*, the system will recommend one to users who liked the other.

### Advantages
- Can discover unexpected relationships between items
- Does not require item metadata
- Often produces more diverse recommendations

### Limitations
- Suffers from cold-start problems for new users and new items
- Performance drops with sparse data
- Recommendations can be hard to explain

---

## 3. Popularity-Based Recommendation System

A popularity-based system recommends items that are **globally popular**, regardless of individual user preferences.

### How it works
- Ranks items by metrics such as:
  - Number of ratings
  - Average rating
  - Number of views or purchases
- Recommends the top-ranked items to all users

### Example
Books with the highest average ratings or most reviews are recommended to everyone.

### Advantages
- Very simple to implement
- Works well for new users (cold-start users)
- Stable and predictable

### Limitations
- Not personalized
- Encourages popularity bias
- Does not adapt to individual tastes

---

## 4. Hybrid Recommendation System

A hybrid recommendation system combines **two or more recommendation techniques** to overcome individual limitations.

### Common combinations
- Content-based + Collaborative filtering
- Collaborative filtering + Popularity-based
- All three combined

### How it works
- Uses collaborative filtering when sufficient user data exists
- Falls back to content-based or popularity-based methods for new users or items
- Produces more robust and reliable recommendations

### Advantages
- Handles cold-start problems better
- Improves recommendation quality
- More flexible and scalable

### Limitations
- More complex to design and maintain
- Requires more data and computation

---

## Summary

| Type | Based On | Personalization | Cold-Start Handling |
|----|----|----|----|
| Content-Based | Item features | High | Good for items |
| Collaborative Filtering | User behavior | High | Poor |
| Popularity-Based | Global trends | None | Excellent |
| Hybrid | Multiple sources | Very High | Best |

In practice, most real-world recommendation systems use **hybrid approaches** to balance accuracy, diversity, and scalability.


# IMPORT LIBS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 

# DATA COLLECTION AND PREPROCESSING

In [2]:
# Loading Data From Csv to pandas dataframe
df = pd.read_csv("../data/raw/movies.csv")
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
# Number of Rows and Columns in the dataset
df.shape

(4803, 24)

# FEATURE SELECTION

In [4]:
# Selecting Important Features
selected_features = ["genres", "keywords", "tagline", "cast", "director"]
print(f"selected_features {selected_features}")
print(f"Normal Features {df.columns}")

selected_features ['genres', 'keywords', 'tagline', 'cast', 'director']
Normal Features Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')


## Reasons for Selected Features

The features `genres`, `keywords`, `tagline`, `cast`, and `director` were selected because they best capture the **semantic and preference-driven characteristics** of movies that are most relevant for a content-based recommendation system.

These features describe *what a movie is about* and *who is involved*, which aligns directly with how users perceive similarity between movies.

---

### 1. Genres

**Reason for selection:**
- Genres represent the primary thematic classification of a movie.
- Users often have strong and consistent genre preferences (e.g., action, romance, thriller).

**Why it matters for recommendations:**
- Movies sharing genres are more likely to appeal to the same audience.
- Genre overlap provides a strong baseline for similarity.

---

### 2. Keywords

**Reason for selection:**
- Keywords capture fine-grained concepts, themes, and narrative elements.
- They go beyond broad genre labels and describe specific plot elements.

**Why it matters for recommendations:**
- Allows the model to distinguish between movies within the same genre.
- Improves recommendation precision by capturing nuanced similarities.

---

### 3. Tagline

**Reason for selection:**
- Taglines are short, descriptive phrases that summarize the movie’s core idea or tone.
- They often highlight the central conflict or emotional appeal.

**Why it matters for recommendations:**
- Adds contextual and thematic information in natural language form.
- Helps capture tone and style similarities between movies.

---

### 4. Cast

**Reason for selection:**
- Actors strongly influence audience preferences.
- Many users follow specific actors across different movies.

**Why it matters for recommendations:**
- Movies sharing cast members are often perceived as related.
- Helps personalize recommendations based on actor affinity.

---

### 5. Director

**Reason for selection:**
- Directors have distinct storytelling styles and thematic preferences.
- Viewers often associate quality and style with directors.

**Why it matters for recommendations:**
- Captures creative consistency across movies.
- Strong signal for stylistic similarity.

---

## Why Other Features Were Not Selected

Although the dataset contains many additional features, they were excluded for the following reasons:

- **Numerical features** (e.g., `budget`, `revenue`, `runtime`, `vote_count`):
  - Measure popularity or scale, not content similarity.
  - Better suited for popularity-based or ranking models.

- **Administrative metadata** (e.g., `id`, `homepage`, `status`):
  - Do not contribute to semantic similarity.

- **Language and production features** (e.g., `original_language`, `production_countries`):
  - Can introduce noise unless explicitly modeling regional preferences.

- **Overview**:
  - Although informative, it is long and unstructured.
  - Requires heavier NLP processing and can dominate other features if not carefully weighted.

---

## Summary

The selected features focus on **content relevance, semantic meaning, and user perception of similarity**. Together, they provide a balanced representation of:
- What the movie is about (genres, keywords, tagline)
- Who created and performed it (director, cast)

This makes them well-suited for building an effective content-based recommendation system.


In [5]:
print(df[selected_features].isnull().sum())
print(df.shape)

genres       28
keywords    412
tagline     844
cast         43
director     30
dtype: int64
(4803, 24)


In [6]:
for features in selected_features : 
    df[features] = df[features].fillna("")

In [7]:
df[selected_features].isnull().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [8]:
combined_movies = df[selected_features]
df["combined_features"] = combined_movies.apply(
    lambda row: " ".join(row.values.astype(str)),
    axis=1
)


In [9]:
df["combined_features"] = df["combined_features"].str.lower()
df["combined_features"]

0       action adventure fantasy science fiction cultu...
1       adventure fantasy action ocean drug abuse exot...
2       action adventure crime spy based on novel secr...
3       action crime drama thriller dc comics crime fi...
4       action adventure science fiction based on nove...
                              ...                        
4798    action crime thriller united states\u2013mexic...
4799    comedy romance  a newlywed couple's honeymoon ...
4800    comedy drama romance tv movie date love at fir...
4801      a new yorker in shanghai daniel henney eliza...
4802    documentary obsession camcorder crush dream gi...
Name: combined_features, Length: 4803, dtype: object

## Vectorizing Words

Vectorization is the process of converting textual data into numerical representations that machine learning models can understand and process. Since algorithms cannot work directly with raw text, vectorization acts as the bridge between human language and mathematical computation.

In this project, vectorization is used to transform combined movie metadata—such as genres, keywords, taglines, cast, and director—into numerical feature vectors. Each movie is treated as a document, and the words describing it are converted into numbers that reflect their importance.

A common approach for vectorization is **TF-IDF (Term Frequency–Inverse Document Frequency)**. TF-IDF assigns higher weights to words that are frequent in a specific document but rare across the entire dataset. This helps the model focus on meaningful and distinctive terms rather than common, less informative words.

Once vectorized, each movie becomes a point in a high-dimensional space. These vectors can then be compared using similarity measures (such as cosine similarity) to identify movies with similar content. This is the foundation of content-based recommendation systems, where items are recommended based on how closely their feature vectors align.


In [10]:
vectorizer = TfidfVectorizer()

In [11]:
df["combined_features"].shape

(4803,)

In [12]:
feature_vectors = vectorizer.fit_transform(df["combined_features"])
feature_vectors

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>

In [13]:
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

### Cosine Similarities 

In [14]:
similarity = cosine_similarity(feature_vectors)

In [15]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [16]:
print(similarity.shape)

(4803, 4803)


In [34]:
# Getting The Movie name from the user
movie_name = input("Enter Movie Name")

Enter Movie Name Dirty Shades of Grey


In [35]:
df["title"].head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: title, dtype: object

In [36]:
# Finding Movies List
list_of_all_movies = df["title"].tolist()
# print(list_of_all_movies)

In [37]:
# Getting The Close Match
import difflib

find_close_match = difflib.get_close_matches(movie_name, list_of_all_movies)

In [38]:
find_close_match

['Fifty Shades of Grey', 'Forty Shades of Blue', 'Fifty Shades of Black']

In [39]:
close_match = find_close_match[0]
print(close_match)

Fifty Shades of Grey


In [40]:
index_of_the_movie  = df[df.title == close_match]["index"].values[0]
print(index_of_the_movie)

1154


In [41]:
# Getting list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, np.float64(0.02118369831917985)), (1, np.float64(0.0)), (2, np.float64(0.06357850080110516)), (3, np.float64(0.009367538295342304)), (4, np.float64(0.0634575494595464)), (5, np.float64(0.021944652958304448)), (6, np.float64(0.0)), (7, np.float64(0.024252258586183982)), (8, np.float64(0.0)), (9, np.float64(0.02369446595196139)), (10, np.float64(0.0)), (11, np.float64(0.004806710521267999)), (12, np.float64(0.0)), (13, np.float64(0.0)), (14, np.float64(0.03443333986444958)), (15, np.float64(0.0391042910679701)), (16, np.float64(0.022996296441003265)), (17, np.float64(0.0)), (18, np.float64(0.018230217954462346)), (19, np.float64(0.0)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.0)), (23, np.float64(0.07330832215446356)), (24, np.float64(0.002810127722197466)), (25, np.float64(0.026096888989506538)), (26, np.float64(0.0)), (27, np.float64(0.02698382869375678)), (28, np.float64(0.0054114730589372535)), (29, np.float64(0.03875977710801707)), (30, np.float64(0.02134

In [42]:
# Sorting The Movies based on their similarities
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True)
print(sorted_similar_movies)

[(1154, np.float64(0.9999999999999998)), (4456, np.float64(0.18787126747150174)), (4171, np.float64(0.16296000300168634)), (1335, np.float64(0.15582491804835555)), (877, np.float64(0.13437604142644177)), (4750, np.float64(0.13206969248728953)), (1644, np.float64(0.12393418078848863)), (2234, np.float64(0.11881577594439151)), (1007, np.float64(0.11403947534696392)), (2984, np.float64(0.10536770767539999)), (3497, np.float64(0.10159475386475525)), (3996, np.float64(0.0999649237033528)), (1075, np.float64(0.09547537593821293)), (3432, np.float64(0.09369467257611744)), (2539, np.float64(0.09358578076688367)), (3066, np.float64(0.09343947738064973)), (914, np.float64(0.08878792608894265)), (4551, np.float64(0.08733076013976257)), (451, np.float64(0.08724769559026885)), (1916, np.float64(0.08716393611627868)), (3319, np.float64(0.08666671459500769)), (1740, np.float64(0.08394120700908003)), (1162, np.float64(0.08348652853787022)), (3325, np.float64(0.08053165900841207)), (1185, np.float64(0.

In [43]:
print("Print Movies For You")

i = 1

for movie in sorted_similar_movies :
    index = movie[0]
    title_from_index = df[df.index == index]["title"].values[0]
    if(i<30): 
        print(f"{i}. {title_from_index}")
        i+=1

Print Movies For You
1. Fifty Shades of Grey
2. Raising Victor Vargas
3. Nowhere Boy
4. How to Be Single
5. Black Mass
6. The Dirties
7. Pride and Glory
8. A Dangerous Method
9. Anna Karenina
10. Madea's Family Reunion
11. The Greatest
12. A Guy Named Joe
13. Savages
14. The Work and the Glory II: American Zion
15. Snitch
16. Black or White
17. Central Intelligence
18. Now Is Good
19. The Haunting
20. Lords of Dogtown
21. The Work and the Glory
22. Kick-Ass 2
23. He's Just Not That Into You
24. Albert Nobbs
25. The Other Woman
26. Moneyball
27. Indignation
28. The Sound and the Shadow
29. Road to Perdition
